In [58]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
import requests
from bs4 import BeautifulSoup

# Send a request to the website
url = 'https://europa.eu/eures/portal/jv-se/search?page=2&resultsPerPage=10&orderBy=BEST_MATCH&locationCodes=fi&lang=en'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all job postings on the page
job_postings = soup.find_all('article', class_='jv-card')

# List to hold job data
jobs_data = []

# Loop through each job posting
for job in job_postings:
    # Extract the job title
    title_tag = job.find('a', class_='ecl-link--standalone')
    title = title_tag.get_text(strip=True) if title_tag else 'Title not found'

    # Extract the location
    location_tag = job.find('span', class_='jv-result-location-country')
    location = location_tag.get_text(strip=True) if location_tag else 'Location not found'

    # Extract the company name
    company_tag = job.find('div', class_='jv-result-summary-title')  # You might need to adjust this selector
    company = company_tag.get_text(strip=True) if company_tag else 'Company not found'

    # Add job info to the list
    jobs_data.append({
        'title': title,
        'location': location,
        'company': company
    })

# Display the results
for job in jobs_data:
    print(job)


In [6]:
jobs_daat

[]

In [6]:
import requests
from bs4 import BeautifulSoup

# URL of the first page of the job listings
url = 'https://europa.eu/eures/portal/jv-se/search?page=1&resultsPerPage=20&orderBy=BEST_MATCH&locationCodes=fi&lang=en'

# Send a request to the website
response = requests.get(url)

# Initialize a variable to store the HTML content if the request is successful
html_content = None

# Check if the request was successful
if response.status_code == 200:
    html_content = response.content
else:
    print(f"Failed to retrieve data: Status code {response.status_code}")

html_content[:50000]  # Display the first 500 characters of the HTML content for inspection


b'<!DOCTYPE html><html lang="en"><head>\n        <meta charset="utf-8">\n        <base href="/eures/portal/">\n        <meta http-equiv="Cache-control" content="no-cache, no-store, must-revalidate">\n        <meta http-equiv="Pragma" content="no-cache">\n        <meta http-equiv="Expires" content="0">\n        <meta name="viewport" content="width=device-width, initial-scale=1">\n        <link rel="icon" type="image/x-icon" href="favicon.ico">\n        <title></title>\n    <style>@charset "UTF-8";\n/*! @ecl/preset-ec - 3.3.0 Built on 2022-05-17T06:21:37.891Z */\n/*!\n * Pikaday\n * Copyright \xc2\xa9 2014 David Bushell | BSD & MIT license | https://dbushell.com/\n */\n/*! normalize.css v8.0.1 | MIT License | github.com/necolas/normalize.css */html{line-height:1.15;-webkit-text-size-adjust:100%}body,div,html{border:0;font:inherit;font-size:100%;margin:0;padding:0}body{line-height:1}:root{--eui-base-color-grey-100:#404040;--eui-base-color-grey-100-rgb:#404040;--eui-base-color-grey-100-con

In [33]:
from bs4 import BeautifulSoup
import requests

# The URL you provided
url = 'https://europa.eu/eures/portal/jv-se/search?page=1&resultsPerPage=10&orderBy=BEST_MATCH&locationCodes=fi&lang=en'

# Send a GET request to the website
response = requests.get(url)

# If the response was successful, parse the HTML
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the h3 tag with the specific class
    h3_tag = soup.find('h3', class_='ecl-u-type-prolonged-m ecl-u-type-bold ecl-u-mb-none')
    
    # Check if the h3 tag was found
    if h3_tag:
        # Find the 'a' tag within it
        job_title_tag = h3_tag.find('a')
        # Extract the text from the 'a' tag
        if job_title_tag:
            job_title = job_title_tag.get_text(strip=True)
            print(job_title)
        else:
            print("Job title 'a' tag not found within the h3 tag.")
    else:
        print("No h3 tag with the specified class found.")

else:
    print(f"Error fetching the page: {response.status_code} - {response.reason}")


No h3 tag with the specified class found.


In [53]:
url = 'https://europa.eu/eures/portal/jv-se/search?page=1&resultsPerPage=20&orderBy=BEST_MATCH&locationCodes=fi&lang=en'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
cards = soup.find_all('span', 'jv-result-employer')

In [54]:
cards

[]

In [90]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_page(offset):
    url = f'https://cv.ee/et/search?countries%5B0%5D=1&params%5B0%5D=country&params%5B1%5D=eesti&offset={offset}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    job_listings = soup.find_all('div', class_='vacancy-item__info')

    results = []
    for job in job_listings:
        title_element = job.find('span', class_='vacancy-item__title')
        title = title_element.get_text().strip() if title_element else 'No Title Found'

        employer_element = job.find('a', class_='jsx-3024910437')
        employer = employer_element.get_text().strip() if employer_element else 'No Employer Found'

        salary_element = job.find('span', class_='vacancy-item__salary-label')
        salary = salary_element.get_text().strip() if salary_element else 'No Salary Info'

        location_element = job.find('div', class_='vacancy-item__column vacancy-item__locations')
        if location_element:
            span_element = location_element.find('span', class_='hide-mobile dashed-span')
            location = span_element.next_sibling.strip() if span_element and span_element.next_sibling else 'No Location Info'
        else:
            location = 'No Location Info'




        results.append({'Title': title, 'Employer': employer, 'Salary': salary, 'Location': location})

    return results

def scrape_all_pages(max_pages):
    all_results = []
    for i in range(max_pages):
        offset = i * 20
        page_results = scrape_page(offset)
        all_results.extend(page_results)
    return all_results

# Scrape data
max_pages_to_scrape = 80  # Adjust this number as needed
data = scrape_all_pages(max_pages_to_scrape)

# Convert to pandas DataFrame
df = pd.DataFrame(data)


In [91]:
df = pd.DataFrame(data)



In [102]:
cvdf = df.drop(columns=['Location'])

In [105]:
import pandas as pd

# Assuming df is your DataFrame and 'Salary' is the column with salary data
# Example: Salary data is in the format '€2000 – €3000'

# Remove currency symbols and split the range
df['Salary'] = df['Salary'].str.replace('€', '').str.split(' – ')

# Convert to average of the range
df['Salary'] = df['Salary'].apply(lambda x: (float(x[0]) + float(x[1])) / 2 if isinstance(x, list) else float(x))

# Now df['Salary'] is in a numeric format


ValueError: could not convert string to float: 'No Salary Info'

In [83]:
url = 'https://europa.eu/eures/portal/jv-se/search?page=1&resultsPerPage=10&orderBy=BEST_MATCH&locationCodes=fi&lang=en'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
job_listings = soup.find_all('div', class_='jv-result-summary-sections')

results = []
for job in job_listings:
        

    employer_element = job.find('span', class_='jv-result-employer-name')
    employer = employer_element.get_text().strip() if employer_element else 'No Employer Found'

    results.append({'Employer': employer})



data = results

# Convert to pandas DataFrame
df = pd.DataFrame(data)


In [86]:
df

""
